In [ ]:
import numpy as np
import argparse
import cv2
import os

from skimage.io import imsave

In [ ]:
for cls in ['Real', 'Fake', 'Test']:
    for path in os.listdir('Video/' + cls):
        vs = cv2.VideoCapture(os.path.join('Video', cls, path))
        read = 0
        saved = 0
        print(os.path.join('Video', cls, path))

        while True:
            (grabbed, frame) = vs.read()

            if not grabbed:
                break

            read += 1

            if read >= 20 and read <= 220:
                rgb_frame = frame[:, :, ::-1]
                if not os.path.exists('Dataset/' + cls + '/' + path[:-4]):
                    os.mkdir('Dataset/' + cls + '/' + path[:-4])
                imsave('Dataset/' + cls + '/' + path[:-4] + '/' + str(read) + '.png', rgb_frame)

In [ ]:
eps = 20

def A_feature(img1, img2):
    return np.abs(img1 - img2)

def A_feature_map(imgs):
    diff = np.array([])
    for i in range(len(imgs)-1):
        diff = np.append(diff, A_feature(imgs[i], imgs[i+1]))
    diff = diff.reshape(shape)
    return diff

def F_feature(img1, img2):
    return (np.abs(img1 - img2) > eps).astype(int) * 255

def F_feature_map(imgs):
    diff = np.array([])
    for i in range(len(imgs)-1):
        diff = np.append(diff, F_feature(imgs[i], imgs[i+1]))
    diff = diff.reshape(shape)
    return diff

In [ ]:
for cls in ['Real', 'Fake', 'Test']:
    for path in os.listdir('Dataset/' + cls):
        print(cls, path)
        if os.path.exists(os.path.join('maps', cls + "_" +path+".npy")):
            continue
        diff = np.array([])
        shape = 0
        for file in range(20, 120):
            if os.path.exists(os.path.join('Dataset/', cls, path, str(file) + '.png')):
                if file != 20:
                    prev_img = curr_img
                curr_img = cv2.imread(os.path.join('Dataset/', cls, path, str(file) + '.png'))
                if file != 20:
                    diff = np.append(diff, F_feature(prev_img, curr_img))
                shape = curr_img.shape
        shape = tuple([-1] + list(shape))
        diff = diff.reshape(shape)
        new_data = np.array([])
        for img in diff:
            new_img = cv2.resize(img, (256, 256))
            new_data = np.append(new_data, new_img)
        new_data = new_data.reshape(-1, 256, 256, 3)
        np.save(os.path.join('maps', cls + "_" +path+".npy"), new_data)

In [ ]:
true_data = np.array([])
for path in os.listdir('maps'):
    if path.split("_")[0] == 'Real':
        true_data = np.append(true_data, np.load('maps/' + path))

In [ ]:
true_data = true_data.reshape(-1, 256, 256, 3)

In [ ]:
false_data = np.array([])
for path in os.listdir('maps'):
    if path.split("_")[0] == 'Fake':
        false_data = np.append(false_data, np.load('maps/' + path))

In [ ]:
false_data = false_data.reshape(-1, 256, 256, 3)

In [ ]:
true_test = np.array([])
for path in os.listdir('maps'):
    if path.split("_")[0] == 'Test' and path.split("_")[1] == 'true':
        true_test = np.append(true_test, np.load('maps/' + path))

In [ ]:
false_test = np.array([])
for path in os.listdir('maps'):
    if path.split("_")[0] == 'Test' and path.split("_")[1] == 'false':
        false_test = np.append(false_test, np.load('maps/' + path))

In [ ]:
false_test = false_test.reshape(-1, 256, 256, 3)
true_test = true_test.reshape(-1, 256, 256, 3)

In [ ]:
true_test.shape, false_test.shape

In [ ]:
import os
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt


import torchvision
import torch.utils.data as data
import torchvision.models as models
import torchvision.transforms as transforms

import PIL
from PIL import Image
from torch.utils.data import DataLoader

import warnings
warnings.filterwarnings('ignore')
import numpy as np

In [ ]:
X = np.append(true_data, false_data).reshape(-1, 256, 256, 3)
Y = [1] * len(true_data) + [0] * len(false_data)

In [ ]:
true_data = []
false_data = []

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)

In [ ]:
X = []
y = []

In [ ]:
class CustomDataset(data.Dataset):

    def __init__(self, root, flist, transform, X, y):
        self.root = root
        self.imlist = flist
        self.transform = transform
        self.X = X
        self.y = y

    def __getitem__(self, index):
        img, target = Image.fromarray(self.X[index], 'RGB'), self.y[index]
        img = self.transform(img)
        return img, target

    def __len__(self):
        return len(self.y)

In [ ]:
from torchvision import transforms

transform_for_train_and_val = transforms.Compose(
    [   
        transforms.ToTensor(),
    ])


batch_size=1

In [ ]:
trainset = CustomDataset('./data/train/train', 'dd', transform_for_train_and_val, X_train, y_train)
valset = CustomDataset('./data/train/train', 'dd', transform_for_train_and_val, X_test, y_test)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, num_workers=8, shuffle=True, pin_memory=True)                          
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, num_workers=8, pin_memory=True)

In [ ]:
import torchvision.models as models

In [ ]:
custom_num_classes = 2

model = models.squeezenet1_1(pretrained=True)
model.classifier = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Conv2d(512, custom_num_classes, kernel_size=1),
    nn.ReLU(inplace=True),
    nn.AvgPool2d(13)
)
model.forward = lambda x: model.classifier(model.features(x)).view(x.size(0), custom_num_classes)

In [ ]:
model.cuda()

In [ ]:
num_classes = 2

batch_size = 1

feature_extract = True

In [ ]:
params_to_update = model.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

optimizer = optim.SGD(params_to_update, lr=0.0001, momentum=0.91)

In [ ]:
init_model = torch.load('final_model.pth')

In [ ]:
model.load_state_dict(init_model['state_dict'])

In [ ]:
model.eval()

In [ ]:
import tqdm

In [ ]:
num_epochs = 25

In [ ]:
def run_epoch(epoch, is_train):
  """
  Training and evaluaton loop over samples
  Args:
      train_mode (bool): True for train mode
  """
    if is_train:
        model.train()
        loader = trainloader
        print("Training epoch: ", epoch + 1, "/", num_epochs)
    else:
        model.eval()
        loader = valloader
        print('Validation')
      
    running_loss = 0.0
    correct = 0.0
    total = 0.0

    for i, data in tqdm.tqdm_notebook(enumerate(loader)):
        images, labels = data

        images, labels = images.cuda(), labels.cuda()

        outputs = model(images)
        loss = criterion(outputs, labels)

        if is_train:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        running_loss += loss.item()
        total += images.data.size(0)

        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels.data).sum()
      
    print('Loss: {:.3f}, accuracy: {:.3f}'.format(running_loss / (i + 1), correct / total * 100.0))

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
for epoch in range(num_epochs):
  #training
    run_epoch(epoch, is_train=True)

  #validation
    with torch.no_grad():
        run_epoch(epoch, is_train=False) 

    state = {
          'epoch': epoch + 1,
          'state_dict': model.state_dict(),
          'optimizer' : optimizer.state_dict()
          }

    torch.save(state, str(epoch) + '_final_model.pth')

In [ ]:
import sklearn
from torch.autograd import Variable

In [ ]:
import tqdm

y_pred = []
y_probas = []

#val
for x in X_test:
    img = Image.fromarray(x, "RGB")
    img = transform_for_train_and_val(img)
    im = Variable(img, requires_grad=True).unsqueeze(0)
    y_pred.append(int(model(im.cuda()).argmax().cpu().numpy()))
    y_probas.append(F.softmax(model(im.cuda()))[0][1].cpu().detach().numpy())

#test
# y_test = [0]*len(false_test)+[1]*len(true_test)
# for x in false_test:
#     img = Image.fromarray(x, "RGB")
#     img = transform_for_train_and_val(img)
#     im = Variable(img, requires_grad=True).unsqueeze(0)
#     y_pred.append(int(model(im.cuda()).argmax().cpu().numpy()))
#     y_probas.append(F.softmax(model(im.cuda()))[0][1].cpu().detach().numpy())
# for x in true_test:
#     img = Image.fromarray(x, "RGB")
#     img = transform_for_train_and_val(img)
#     im = Variable(img, requires_grad=True).unsqueeze(0)
#     y_pred.append(int(model(im.cuda()).argmax().cpu().numpy()))
#     y_probas.append(F.softmax(model(im.cuda()))[0][0].cpu().detach().numpy())

In [ ]:
sklearn.metrics.precision_recall_fscore_support(y_test, y_pred)

In [ ]:
def get_far_frr(y_true, y_pred, threshold):
    far = sum([1 for i in range(len(y_pred)) if y_pred[i] >= threshold and y_true[i]==0])/(len(y_true)-sum(y_true))
    frr = sum([1 for i in range(len(y_pred)) if y_pred[i] <= threshold and y_true[i]==1])/sum(y_true)
    return far, frr

In [ ]:
def get_far_frr_lists(y_true, y_pred, thresholds=np.arange(0, 1.01, 0.01)):
    far_list = []
    frr_list = []
    for t in thresholds:
        far, frr = get_far_frr(y_true, y_pred, t)
        far_list.append(far)
        frr_list.append(frr)
    return far_list, frr_list


In [ ]:
far, frr = get_far_frr_lists(y_test, y_probas)

In [ ]:
def eer(far_list, frr_list, thresholds):
    eer_1 = far_list[np.argmin(np.absolute((np.asarray(far_list) - np.asarray(frr_list))))]
    eer_2 = frr_list[np.argmin(np.absolute((np.asarray(far_list) - np.asarray(frr_list))))]
    return eer_1, eer_2, thresholds[np.nanargmin(np.absolute((np.asarray(far_list) - np.asarray(frr_list))))]

In [ ]:
eer(far, frr, np.arange(0, 1.01, 0.01))

In [ ]:
import matplotlib

In [ ]:
def DETCurve(far, frr, ticks = np.arange(0.00, 0.31, 0.01)):
    plt.rcParams["figure.figsize"] = [15, 10]
    plt.plot(far, frr, linestyle='-', color='blue', linewidth=2, marker=".", markersize=10)
    plt.plot(ticks, ticks, color='red', linestyle='--', linewidth=1)
    plt.xlim((0, 0.3))
    plt.ylim((0, 0.3))
    plt.xticks(ticks, labels=[str(i) +'%' for i in range(0, 31)], size=15)
    plt.yticks(ticks, labels=[str(i) +'%' for i in range(0, 31)], size=15)
    plt.ylabel("Доля ложных доступов (FAR)", fontsize=20)
    plt.xlabel("Доля ложных отказов (FRR)", fontsize=20)
    plt.title("Ошибки модели на разных порогах", fontsize=25)
    plt.grid(True, which='both', ls="-", color='0.65')

In [ ]:
DETCurve(far, frr)
# plt.savefig("result.png")